N--Normal
I--Imbalance

S1--sensor1
S2--sensors2

H--Horizontal HORIZ
V--Vertical VERT
A--Axial AXIAL

V--Velocity
A--Acceleration 
AP--AccelerationP2P

ex：NS1HV--Normal sensor1 Horizontal Velocity
sp：Horizontal has no Acceleration

Merge folder files

In [17]:
import os
import pandas as pd


folder_path = '/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Rotating Machinery Repository - Faulty/Sensor 2/Vibration - Vertical/velocityRMS'

# Initialise an empty DataFrame to store the data
dfs = []


for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  
        file_path = os.path.join(folder_path, filename)
        
        
        df = pd.read_csv(file_path)
        dfs.append(df)

# Use the concat function to combine all data frames into one
merged_df = pd.concat(dfs, ignore_index=True)

# 
output_file = '/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Rotating Machinery Repository - Faulty/Sensor 2/IS2VV.csv'

# 
merged_df.to_csv(output_file, index=False)  # 


Merge data for each feature, 8 columns, AAP, AA, AV, HAP, HV, VAP, VA, VV

In [29]:
import os
import pandas as pd

# Define the directory containing the CSV files
directory_path = "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2"

# Initialize an empty DataFrame for the combined data
combined_columns_df = pd.DataFrame()

# Loop through each file in the directory
for file in os.listdir(directory_path):
    # Check if the file is a CSV file
    if file.endswith(".csv"):
        file_path = os.path.join(directory_path, file)
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if the DataFrame has at least 2 columns
        if df.shape[1] < 2:
            continue  # Skip files with less than 2 columns
        
        # Extract the second column and rename it
        second_col = df.iloc[:, 1]
        second_col.name = file.replace('.csv', '')  # Use the file name (without .csv) as the column name
        
        # Add the second column to the combined DataFrame
        combined_columns_df = pd.concat([combined_columns_df, second_col], axis=1)

# Define the path for the new combined CSV file
combined_file_path = "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/combined_N_S2.csv"

# Save the combined DataFrame to a new CSV file
combined_columns_df.to_csv(combined_file_path, index=False)


Delete data smaller than the mean value of the first column, simulate switching off the computer, and save the filtered data to a new CSV file

In [31]:
import pandas as pd

# Define the path to the original CSV file
file_path = "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/combined_N_S2.csv"

# Reading CSV files
df = pd.read_csv(file_path)

# Calculate the mean of the first column
first_col_mean = df.iloc[:, 0].mean()

# Filter out rows where the first column is greater than or equal to the mean value
filtered_df = df[df.iloc[:, 0] >= first_col_mean]

# Define the path to the new CSV file to be saved
filtered_file_path = "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/filtered_N_S2.csv"

# Save filtered data to a new CSV file
filtered_df.to_csv(filtered_file_path, index=False)

print(f"Filtered data has been saved to {filtered_file_path}")


Filtered data has been saved to /Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/filtered_N_S2.csv


Combine multiple documents according to the first column of time,Inconsistency in the timing of data collection

In [ ]:

import pandas as pd

# File paths for the uploaded files
file_paths_ns = {
    "NS2AA": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Axial/NS2AA.csv",
    "NS2AAP": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Axial/NS2AAP.csv",
    "NS2AV": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Axial/NS2AV.csv",
    "NS2HAP": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Horizontal/NS2HAP.csv",
    "NS2HV": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Horizontal/NS2HV.csv",
    "NS2VA": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Vertical/NS2VA.csv",
    "NS2VAP": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Vertical/NS2VAP.csv",
    "NS2VV": "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Normal/NoFault/Sensor2/Vibration - Vertical/NS2VV.csv"
}

# Load and display the first few rows of each file to understand their structure
data_samples_ns = {name: pd.read_csv(path).head() for name, path in file_paths_ns.items()}
data_samples_ns


# Convert 'date' column to datetime and merge all files based on the 'date' column
data_frames_ns = [pd.read_csv(path).assign(date=lambda df: pd.to_datetime(df['date'], unit='ms')) for path in file_paths_ns.values()]

# Merging all dataframes on the 'date' column
merged_ns = data_frames_ns[0]
for df in data_frames_ns[1:]:
    merged_ns = pd.merge(merged_ns, df, on='date', how='outer')

# Sorting by date and resetting the index
merged_ns = merged_ns.sort_values('date').reset_index(drop=True)

# Displaying the first few rows of the merged dataset
merged_ns.head()


Forward and Backward Filling Missing Values

In [ ]:
# Forward filling and then backward filling to minimize missing data as much as possible
forward_filled_ns = merged_ns.fillna(method='ffill')
backward_filled_ns = forward_filled_ns.fillna(method='bfill')

# Displaying the first few rows of the filled dataset and checking for remaining missing data
filled_ns_info = {
    "head": backward_filled_ns.head(),
    "missing_data": backward_filled_ns.isnull().sum()
}
filled_ns_info


In [ ]:
file_path = "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Fault/Sensor 1/merged_sensor_data.csv"
df = pd.read_csv(file_path)

# Using forward padding for missing data
df_forward_filled = df.fillna(method='ffill')

# Use backward padding to handle the remaining missing data
df_filled = df_forward_filled.fillna(method='bfill')

# Prints the first few rows of the populated data frame
print(df_filled.head())

# If desired, you can save the populated data as a new CSV file
output_file_path = "/Users/zhangcheng/DataspellProjects/ISCF/DataExtract/Fault/Sensor 1/filled_data.csv"
df_filled.to_csv(output_file_path, index=False)

Remove data below the mean

In [ ]:
# Calculating the mean for each column
means = backward_filled_ns.mean()

# Removing rows where all values (excluding 'date') are below their respective column means
filtered_ns = backward_filled_ns[~(backward_filled_ns.loc[:, "NS1AA_rms":] < means).all(axis=1)]

# Displaying the first few rows of the filtered dataset and the new dataset shape
filtered_ns_info = {
    "head": filtered_ns.head(),
    "shape": filtered_ns.shape
}
filtered_ns_info


Equal resampling intervals

In [ ]:
# Determine the most common time interval in the original data
time_diffs = sorted_filtered_ns['date'].diff().dt.total_seconds().mode()

# If multiple modes are found, choose the smallest one as the interval
interval = time_diffs.min()

# Resampling the data to the determined interval
# 'date' needs to be the index for resampling
sorted_filtered_ns.set_index('date', inplace=True)
resampled_ns = sorted_filtered_ns.resample(f'{int(interval)}S').mean().interpolate()

# Resetting index to bring 'date' back as a column
resampled_ns.reset_index(inplace=True)

# Displaying the first few rows of the resampled dataset and the interval used
resampled_info = {
    "head": resampled_ns.head(),
    "interval_seconds": interval
}
resampled_info


another

In [ ]:
# First, let's load and examine the data from the provided files to understand their structure and contents.

import pandas as pd

# Load the files into dataframes
files = [
    "/mnt/data/NS2AA.csv", "/mnt/data/NS2AAP.csv", "/mnt/data/NS2AV.csv",
    "/mnt/data/NS2HAP.csv", "/mnt/data/NS2HV.csv", "/mnt/data/NS2VA.csv",
    "/mnt/data/NS2VAP.csv", "/mnt/data/NS2VV.csv"
]

dataframes = [pd.read_csv(file) for file in files]

# Display the first few rows of each dataframe to understand their structure
for i, df in enumerate(dataframes):
    print(f"Data from file {i+1} ({files[i]}):")
    print(df.head(), "\n")


In [ ]:
# Converting the timestamp from integer to datetime format
for df in dataframes:
    df['date'] = pd.to_datetime(df['date'], unit='ms')

# Re-trying the merge with the converted datetime
merged_df = dataframes[0]

for df in dataframes[1:]:
    merged_df = pd.merge_asof(merged_df.sort_values('date'), df.sort_values('date'), on='date', tolerance=pd.Timedelta('1min'), direction='nearest')

# Renaming columns for clarity
column_names = ['date', 'AA_rms', 'AAP_amplitude', 'AV_rms', 'HAP_amplitude', 'HV_rms', 'VA_rms', 'VAP_amplitude', 'VV_rms']
merged_df.columns = column_names

# Displaying the first few rows of the merged dataframe
merged_df.head()


In [ ]:
# Excluding the 'date' column for mean calculation and filtering
numeric_df = filled_df.drop(columns=['date'])

# Calculating the mean for each numeric column
means = numeric_df.mean()

# Filtering out the rows where all values are below their respective means
filtered_df = filled_df[(numeric_df > means).any(axis=1)]

# Exporting the final filtered data to a new CSV file
final_filtered_file_path = "/mnt/data/final_filtered_sensor_data.csv"
filtered_df.to_csv(final_filtered_file_path, index=False)

final_filtered_file_path
